In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('code_jeje_yang/Submit/images/DZ6079a_02A05 DAPI 10x.tif', 0)
edges1 = cv2.Canny(img, 10, 100)
edges2 = cv2.Canny(img, 10, 150)
cv2.imshow('edges1', edges2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
horizon_kernel = np.ones((1,3), np.uint8)
vertical_kernel = np.ones((3,1), np.uint8)

In [8]:
dilation = cv2.dilate(edges2,horizon_kernel,iterations = 1)
dilation = cv2.dilate(dilation,vertical_kernel,iterations = 1)
cv2.imshow('dilation', dilation)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
imagem = cv2.bitwise_not(dilation)
cv2.imshow('imagem', imagem)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
h, w = imagem.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)

flood = cv2.floodFill(imagem, mask, (0,0), 255)
print(flood[1].shape)
cv2.imshow('flood', flood[1])

cv2.waitKey(0)
cv2.destroyAllWindows()

(1040, 1388)
